**INITIALIZATION**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [3]:
#@ INSTALLING DEPENDENCIES: UNCOMMENT BELOW: 
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [4]:
#@ DOWNLOADING LIBRARIES AND DEPENDENCIES: 
from fastbook import *                                  # Getting all the Libraries. 
from fastai.callback.fp16 import *

**MODELING NEURON**

In [5]:
#@ MODELING NEURON FROM SCRATCH: UNCOMMENT BELOW: 
# output = sum([x*w for x,w in zip(inputs, weights)]) + bias          # Adding Weighted Inputs and Bias. 
# def relu(x): return x if x >= 0 else 0                              # Defining RELU Activation Function. 
# y[i, j] = sum([a*b for a,b in zip(x[i,:], w[j,:])]) + b[j]          # Initializing Matrix Multiplication. 

In [8]:
#@ MATRIX MULTIPLICATION FROM SCRATCH: 
def matmul(a, b):                            # Defining Matrix Multiplication Function. 
    ar, ac = a.shape                         # Inspecting Shape. 
    br, bc = b.shape                         # Inspecting Shape. 
    assert ac == br                          # Asserting Rows and Columns. 
    c = torch.zeros(ar, bc)                  # Initializing Tensor. 
    for i in range(ar):                      # Getting Row Indices. 
        for j in range(bc):                  # Getting Column Indices. 
            for k in range(ac):              # Getting Inner Sum. 
                c[i,j] += a[i,k] * b[k,j]    # Getting Matrix Multiplication. 
    return c

#@ IMPLEMENTATION OF MATRIX MULTIPLICATION: 
m1 = torch.randn(5, 28*28)                   # Initializing Matrix. 
m2 = torch.randn(784, 10)                    # Initializing Matrix. 
%time t1 = matmul(m1, m2)                    # Implementation of Matrix Multiplication. 
%timeit -n 20 t2 = m1@m2                     # Implementation of Matrix Multiplication. 

CPU times: user 701 ms, sys: 0 ns, total: 701 ms
Wall time: 701 ms
The slowest run took 168.82 times longer than the fastest. This could mean that an intermediate result is being cached.
20 loops, best of 5: 5.74 µs per loop


In [10]:
#@ ELEMENTWISE ARITHMETIC: 
a = tensor([10., 6, -4])                     # Initializing a Tensor. 
b = tensor([2., 8, 7])                       # Initializing a Tensor. 
a + b                                        # Initializing Elementwise Addition. 
(a<b).all(), (a==b).all()                    # Combining Elementwise Operations. 

(tensor(False), tensor(False))